In [1]:
import time
start = time.time()
import random
import string
import pandas as pd
import copy
#import os
#os.chdir('/home/scott/Documents/EDx/MIT/Python/Spyder_scripts/')
WORDLIST_FILENAME = "words.txt"

def loadWords():
    """
    Returns a list of valid words. Words are strings of lowercase letters.
    
    Depending on the size of the word list, this function may
    take a while to finish.
    """
    print("Loading word list from file...")
    # inFile: file
    inFile = open(WORDLIST_FILENAME, 'r')
    # line: string
    line = inFile.readline()
    # wordlist: list of strings
    wordlist = line.split()
    print("  ", len(wordlist), "words loaded.")
    return wordlist

def chooseWord(wordlist):
    """
    wordlist (list): list of words (strings)

    Returns a word from wordlist at random
    """
    return random.choice(wordlist)

# end of helper code
# -----------------------------------

# Load the list of words into the variable wordlist
# so that it can be accessed from anywhere in the program

def isWordGuessed(secretWord, lettersGuessed):
    '''
    secretWord: string, the word the user is guessing
    lettersGuessed: list, what letters have been guessed so far
    returns: boolean, True if all the letters of secretWord are in lettersGuessed;
      False otherwise
    '''
    for char in secretWord:
        if char not in lettersGuessed:
            return False
    
    return True  


def getGuessedWord(secretWord, lettersGuessed):
    '''
    secretWord: string, the word the user is guessing
    lettersGuessed: list, what letters have been guessed so far
    returns: string, comprised of letters and underscores that represents
      what letters in secretWord have been guessed so far.
    '''
    # FILL IN YOUR CODE HERE...
    word = secretWord
 
    for char in word:
        if char not in lettersGuessed:
            word = word.replace(char,'_')
            
    return word
    


def getAvailableLetters(lettersGuessed):
    '''
    lettersGuessed: list, what letters have been guessed so far
    returns: string, comprised of letters that represents what letters have not
      yet been guessed.
    '''
    # FILL IN YOUR CODE HERE...
    availableLetters = 'abcdefghijklmnopqrstuvwxyz'
    for char in lettersGuessed:
            position = availableLetters.index(char)
            availableLetters = availableLetters[0:position] + availableLetters[position+1::]
    return availableLetters

wordList = loadWords()
columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13']
secrets = pd.DataFrame()
    
for i in range(1,max(len(word) for word in wordList)+1):
    dat = []
    for word in wordList:
        if len(word) == i:
            dat.append(word)
    dat = pd.Series(dat)
    secrets = pd.concat([secrets,dat],axis=1, join='outer', join_axes=None, ignore_index=False)
secrets.columns = columns

def hangman():
    '''
    secretWord: string, the secret word to guess.

    Starts up an interactive game of Hangman.

    * At the start of the game, let the user know how many 
      letters the secretWord contains.

    * Ask the user to supply one guess (i.e. letter) per round.

    * The user should receive feedback immediately after each guess 
      about whether their guess appears in the computers word.

    * After each round, you should also display to the user the 
      partially guessed word so far, as well as letters that the 
      user has not yet guessed.

    Follows the other limitations detailed in the problem write-up.
    '''
    
    n = 1
    gamesWon = 0
    while n <= 100:    
        secretWord = chooseWord(wordList)
        word = secretWord
        choices = secrets[str(len(word))]
        numberOfGuesses = 8
        print('Welcome to the game Hangman!')
        print('I am thinking of a word that is ' + str(len(word)) + ' letters long.')
        print('-------------')
        lettersGuessed = ''
        game = True
        
        while game == True:
            
                
            availableLetters = getAvailableLetters(lettersGuessed)
            print('You have ' + str(numberOfGuesses) + ' guesses left.')
            print('Available letters: ' + availableLetters)
                
            word = getGuessedWord(secretWord, lettersGuessed)
            #letterGuessed = player(secretWord, lettersGuessed)
            [letterGuessed,words] = playerAI(secretWord, lettersGuessed,word,choices)
            choices = words
                
            lettersGuessed = lettersGuessed[0::] + letterGuessed
            word = getGuessedWord(secretWord, lettersGuessed)
                    
            if letterGuessed not in secretWord:
                print("Oops! That letter is not in my word: " + word)
                print('-------------')
                numberOfGuesses -= 1
                    
            else:
                print('Good guess: ' + word)
                print('-------------')
               
            answer = isWordGuessed(secretWord,lettersGuessed)
        
            if numberOfGuesses == 0 or answer == True:
                game = False    
            
            
        if answer == True:
            print('Congratulations, you won!')
            gamesWon += 1
            n += 1
            
        else:
            print('Sorry, you ran out of guesses. The word was ' + secretWord)
            n += 1
            
    print('Player function won ' + str(gamesWon) + ' games.')
    print('Player function lost ' + str(n - gamesWon - 1) + ' games.')
    return n

# When you've completed your hangman function, uncomment these two lines
# and run this file to test! (hint: you might want to pick your own
# secretWord while you're testing)

def playerAI(secretWord, lettersGuessed,word, choices):

    availableLetters = getAvailableLetters(lettersGuessed)
    col = str(len(secretWord))
    words = secrets[col].dropna()
    words = list(words)
    options = copy.deepcopy(words)

    if len(availableLetters) < 26:
        for chance in words:
            for j in range(0,len(chance)):
                if word[j] != '_' and  word[j] != chance[j]:
                    options.remove(chance)
                    break       
            
    best = 0
    choice = ''
    for letter in availableLetters:
        
        maximum = 0
        for word in options:
            for char in word:
                if char == letter:
                    maximum += 1
        if maximum > best:
            best = maximum
            choice = letter

    return [choice,options]
    

def player(secretWord, lettersGuessed):
    
    index = random.choice(range(0,len(string.ascii_lowercase) - len(lettersGuessed)  , 1))
    availableLetters = getAvailableLetters(lettersGuessed)
    letter = availableLetters[index]
    
    return letter

n = hangman()
end = time.time()
print('PlayerAI played ' + str(n-1) + ' games in '+ str(round((end-start)/60,2)) + ' minutes')

Loading word list from file...
('  ', 55909, 'words loaded.')
Welcome to the game Hangman!
I am thinking of a word that is 7 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Good guess: e_e__e_
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Good guess: e_e_ded
-------------
You have 8 guesses left.
Available letters: abcfghijklmnopqrstuvwxyz
Good guess: eme_ded
-------------
You have 8 guesses left.
Available letters: abcfghijklnopqrstuvwxyz
Good guess: emended
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 9 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Good guess: _______e_
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Oops! That letter is not in my word: _______e_
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Oops! Tha

Oops! That letter is not in my word: ________
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Oops! That letter is not in my word: ________
-------------
You have 6 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Good guess: _i_i_i__
-------------
You have 6 guesses left.
Available letters: abcdfghjklmnopqrtuvwxyz
Good guess: _ivi_i__
-------------
You have 6 guesses left.
Available letters: abcdfghjklmnopqrtuwxyz
Good guess: divi_i__
-------------
You have 6 guesses left.
Available letters: abcfghjklmnopqrtuwxyz
Good guess: divini__
-------------
You have 6 guesses left.
Available letters: abcfghjklmopqrtuwxyz
Oops! That letter is not in my word: divini__
-------------
You have 5 guesses left.
Available letters: abcfhjklmopqrtuwxyz
Good guess: divinit_
-------------
You have 5 guesses left.
Available letters: abcfhjklmopqruwxyz
Good guess: divinity
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word

Oops! That letter is not in my word: _________
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Oops! That letter is not in my word: _________
-------------
You have 6 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Good guess: __i___i__
-------------
You have 6 guesses left.
Available letters: abcdfghjklmnopqrtuvwxyz
Good guess: __i___in_
-------------
You have 6 guesses left.
Available letters: abcdfghjklmopqrtuvwxyz
Oops! That letter is not in my word: __i___in_
-------------
You have 5 guesses left.
Available letters: abcdfhjklmopqrtuvwxyz
Good guess: __i_l_in_
-------------
You have 5 guesses left.
Available letters: abcdfhjkmopqrtuvwxyz
Good guess: __i_l_ind
-------------
You have 5 guesses left.
Available letters: abcfhjkmopqrtuvwxyz
Good guess: w_i_lwind
-------------
You have 5 guesses left.
Available letters: abcfhjkmopqrtuvxyz
Good guess: whi_lwind
-------------
You have 5 guesses left.
Available letters: abcfjkmopqrtuvxyz
Good gu

Good guess: _r___s
-------------
You have 6 guesses left.
Available letters: bcdfghijklmnopqtuvwxyz
Oops! That letter is not in my word: _r___s
-------------
You have 5 guesses left.
Available letters: bcdfghijklmnopquvwxyz
Good guess: _ro__s
-------------
You have 5 guesses left.
Available letters: bcdfghijklmnpquvwxyz
Oops! That letter is not in my word: _ro__s
-------------
You have 4 guesses left.
Available letters: bdfghijklmnpquvwxyz
Oops! That letter is not in my word: _ro__s
-------------
You have 3 guesses left.
Available letters: bdfghijklmnquvwxyz
Good guess: _ro_ns
-------------
You have 3 guesses left.
Available letters: bdfghijklmquvwxyz
Oops! That letter is not in my word: _ro_ns
-------------
You have 2 guesses left.
Available letters: bdfghijklmquvxyz
Good guess: gro_ns
-------------
You have 2 guesses left.
Available letters: bdfhijklmquvxyz
Good guess: groins
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 10 lette

Good guess: __e_e__
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Good guess: _re_er_
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqstuvwxyz
Good guess: _re_ers
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqtuvwxyz
Good guess: bre_ers
-------------
You have 8 guesses left.
Available letters: acdfghijklmnopqtuvwxyz
Good guess: brewers
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 8 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Good guess: __e___e_
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Good guess: _re___e_
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqstuvwxyz
Oops! That letter is not in my word: _re___e_
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqtuvwxyz
Good guess: _re_d_ed
----------

...

Oops! That letter is not in my word: ________
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Good guess: _______s
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Good guess: _a_____s
-------------
You have 7 guesses left.
Available letters: bcdfghijklmnopqrtuvwxyz
Good guess: _ar__r_s
-------------
You have 7 guesses left.
Available letters: bcdfghijklmnopqtuvwxyz
Oops! That letter is not in my word: _ar__r_s
-------------
You have 6 guesses left.
Available letters: bdfghijklmnopqtuvwxyz
Good guess: _ar__rds
-------------
You have 6 guesses left.
Available letters: bfghijklmnopqtuvwxyz
Good guess: _arl_rds
-------------
You have 6 guesses left.
Available letters: bfghijkmnopqtuvwxyz
Good guess: _arlords
-------------
You have 6 guesses left.
Available letters: bfghijkmnpqtuvwxyz
Good guess: warlords
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 4 letters long.


Oops! That letter is not in my word: _________
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Good guess: ________s
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Oops! That letter is not in my word: ________s
-------------
You have 6 guesses left.
Available letters: abcdfghjklmnopqrtuvwxyz
Good guess: a_______s
-------------
You have 6 guesses left.
Available letters: bcdfghjklmnopqrtuvwxyz
Oops! That letter is not in my word: a_______s
-------------
You have 5 guesses left.
Available letters: bcdfghjklmnopqruvwxyz
Good guess: a__r____s
-------------
You have 5 guesses left.
Available letters: bcdfghjklmnopquvwxyz
Good guess: a_or__o_s
-------------
You have 5 guesses left.
Available letters: bcdfghjklmnpquvwxyz
Oops! That letter is not in my word: a_or__o_s
-------------
You have 4 guesses left.
Available letters: cdfghjklmnpquvwxyz
Good guess: a_or_ho_s
-------------
You have 4 guesses left.
Available letters: 

Good guess: ______e_
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Good guess: ______es
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Good guess: ___i_ies
-------------
You have 8 guesses left.
Available letters: abcdfghjklmnopqrtuvwxyz
Oops! That letter is not in my word: ___i_ies
-------------
You have 7 guesses left.
Available letters: abcdfghjklmnopqruvwxyz
Oops! That letter is not in my word: ___i_ies
-------------
You have 6 guesses left.
Available letters: abcdghjklmnopqruvwxyz
Oops! That letter is not in my word: ___i_ies
-------------
You have 5 guesses left.
Available letters: bcdghjklmnopqruvwxyz
Oops! That letter is not in my word: ___i_ies
-------------
You have 4 guesses left.
Available letters: bcdghjklmnopquvwxyz
Good guess: _o_i_ies
-------------
You have 4 guesses left.
Available letters: bcdghjklmnpquvwxyz
Good guess: _omi_ies
-------------
You have 4 guesses left.
Available letters: bcdghjkln

Good guess: ____e_ee__
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Good guess: ____e_eer_
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqstuvwxyz
Good guess: __s_e_eers
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqtuvwxyz
Good guess: __ske_eers
-------------
You have 8 guesses left.
Available letters: abcdfghijlmnopqtuvwxyz
Good guess: m_ske_eers
-------------
You have 8 guesses left.
Available letters: abcdfghijlnopqtuvwxyz
Good guess: m_sketeers
-------------
You have 8 guesses left.
Available letters: abcdfghijlnopquvwxyz
Good guess: musketeers
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 8 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Oops! That letter is not in my word: ________
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Oops! That letter 

KeyboardInterrupt: 